In [64]:
from dotenv import load_dotenv
load_dotenv()
import os

In [65]:
from langchain_community.document_loaders import JSONLoader
import json
from pathlib import Path
from pprint import pprint


file_path='scholarship_data.json'
data = json.loads(Path(file_path).read_text())


In [66]:
pprint(data)

{'categories': ['Government of India Post-Matric Scholarship',
                'Post-Matric Tuition Fee and Examination Fee (Freeship)',
                'Maintenance Allowance for student Studying in professional '
                'courses',
                'Rajarshri Chhatrapati Shahu Maharaj Merit Scholarship',
                'Post-Matric Scholarship for persons with disability',
                'Vocational Training Fee reimbursement for the students '
                'belonging to Scheduled Caste category Students',
                'Post Matric Scholarship Scheme (Government Of India )',
                'Tuition Fee &\xa0 Exam Fee for Tribal Students ( Freeship)',
                'Vocational Education Fee Reimbursement',
                'Vocational Education Maintenance Allowance',
                'Vocational Training Fee reimbursement for the students '
                'belonging to Scheduled Tribe Category',
                'Rajarshi Chhatrapati Shahu Maharaj Shikshan Shulkh '
  

In [4]:
data['eligibility']

['1) The parents/Guardian annual income shall be less than or equal to Rs. 2,50,000.2) Student category shall belong to Scheduled caste or Navbouddha3) Student shall belong to resident of Maharashtra 4) Student should be passes SSC/equivalent Matric.  5) Failure: if student fail first time then should avail Exam fees & maintenance allowance. If fails second time in same class then he/she will not get any allowance and after two drops he will pass and go to next class then he will be eligible.6) In case of student studying out of Maharashtra then same rules are applicable as per GOI.7) Only 2 professional Courses are allowed.',
 '• The parents / Guardian annual income above Rs. 250000.to unlimited.\n• Student category should be Scheduled caste or Neo bouddhist\n• Student should be resident of Maharashtra\n• Student should be passes SSC/equivalent Matric.              \n• Institute should be located in Maharashtra & shall be government recognized.\n• For Professional Courses student shou

In [1]:
import json

# Load the existing data
with open('scholarship_data.json') as f:
    data = json.load(f)

# Extract categories and eligibility information
categories = data['categories']
eligibilities = data['eligibility']
links = data['links']

# Create the new schema for scholarship data
scholarship_data = [
    {"name": categories[i], "eligibility": eligibilities[i], "links": links[i]}
    for i in range(len(categories))
]

scholarship_data
with open('scholarship_data_transformed_for_lakshitaa.json', 'w') as json_file:
    json.dump(scholarship_data, json_file, indent=4)

In [68]:
loader = JSONLoader(
    file_path='scholarship_data_transformed.json',
    jq_schema='.[]',
    text_content=False)

data = loader.load()

In [69]:
data

[Document(metadata={'source': 'C:\\Users\\yugtg\\Desktop\\Brainiacs\\Brainiacs-Codeissance\\ml\\scholarship_data_transformed.json', 'seq_num': 1}, page_content='{"name": "Government of India Post-Matric Scholarship", "eligibility": "1) The parents/Guardian annual income shall be less than or equal to Rs. 2,50,000.2) Student category shall belong to Scheduled caste or Navbouddha3) Student shall belong to resident of Maharashtra 4) Student should be passes SSC/equivalent Matric.  5) Failure: if student fail first time then should avail Exam fees & maintenance allowance. If fails second time in same class then he/she will not get any allowance and after two drops he will pass and go to next class then he will be eligible.6) In case of student studying out of Maharashtra then same rules are applicable as per GOI.7) Only 2 professional Courses are allowed."}'),
 Document(metadata={'source': 'C:\\Users\\yugtg\\Desktop\\Brainiacs\\Brainiacs-Codeissance\\ml\\scholarship_data_transformed.json',

In [70]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

In [71]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(data)
vector = FAISS.from_documents(documents, embeddings)

In [72]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0)
prompt = ChatPromptTemplate.from_template("""
Give the five best possible scheme types on which the student can apply for scholarship that matches the user profile like the caste and income and the student's academic performance and if 
the student is a domicile of maharashtra.
The eligibility criteria for the scholarship should match the student's profile as much as possible,preferrably 100%.

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [73]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [62]:
user_profile = """
I am a Scheduled Caste student from Maharashtra. 
My family's annual income is Rs. 260,000. 
I have passed SSC and am looking for scholarship opportunities.
"""

response = retrieval_chain.invoke({"input": "{user_profile}"})
print(response["answer"])


Based on the user profile provided (caste, income, academic performance, domicile in Maharashtra), here are five scholarship schemes that match the criteria:

1. Post-Matric Tuition Fee and Examination Fee (Freeship)
   - Eligibility: Student category should be Scheduled caste or Neo bouddhist, annual income above Rs. 250,000, resident of Maharashtra, passes SSC/equivalent Matric, institute located in Maharashtra, admitted through CAP round for Professional Courses.

2. Vocational Education Fee Reimbursement
   - Eligibility: Applicable for ST only, family annual income limit above Rs. 2,50,000.

3. Scholarship for students of minority communities pursuing Higher and Professional courses(DTE)
   - Eligibility: Nationality of India, Domicile of Maharashtra State, pass SSC from Maharashtra State, admitted for Professional and Technical course, admitted through CAP/Institute Level, total annual family income not more than 8 Lakhs.

4. Maintenance Allowance for student Studying in professi

In [63]:
# import os
# from langchain import hub
# from langchain.agents import AgentExecutor, create_openai_functions_agent
# from langchain_openai import ChatOpenAI
# from langchain.tools.retriever import create_retriever_tool

# instructions = """Give the five best possible scheme types on which the student can apply for scholarship that matches the user profile like the caste and income and the student's academic performance and if 
# the student is a domicile of maharashtra.
# The eligibility criteria for the scholarship should match the student's profile as much as possible,preferrably 100%.
# """
# base_prompt = hub.pull("langchain-ai/openai-functions-template")
# prompt = base_prompt.partial(instructions=instructions)
# llm = ChatOpenAI(api_key= os.getenv("OPENAI_API_KEY") ,temperature=0)

# retriever_tool = create_retriever_tool(
#     retriever,
#     "scholarship_recommender_tool",
#     "For any suggestion about scholarships use this tool compulsarily",
# )
# tools = [retriever_tool]
# agent = create_openai_functions_agent(llm, tools, prompt)
# agent_executor = AgentExecutor(
#     agent=agent,
#     tools=tools,
#     verbose=True,
# )

# user_profile = """
# I am a Scheduled Caste student from Maharashtra. 
# My family's annual income is Rs. 800,000. 
# I have failed SSC and am looking for scholarship opportunities.
# """


# ans = agent_executor.invoke({"input": "{user_profile}"})
# print(ans["output"])

c:\Users\yugtg\Desktop\Brainiacs\Brainiacs-Codeissance\.venv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...

Invoking: `scholarship_recommender_tool` with `{'query': 'Maharashtra domicile, specific caste, income level, academic performance'}`
responded: I see that you are a student from Maharashtra belonging to a specific caste with a certain income level and academic performance. Let me find the best possible scholarship schemes that match your profile.

{"name": "Dr. Panjabrao Deshmukh Vasatigruh Nirvah Bhatta Yojna (MAFSU)", "eligibility": "a). Applicant should have Nationality of India.b). Candidate should be Domicile of Maharashtra State.  c). Applicant should be \u201dBonafide Student of Institute\u201d and admitted for Professional  and Technical course (Graduation / Post Graduation Degree) as mentioned in GR.  d). Deemed University and Private university is not applicable.e). Candidate should be admitted through Centralized Admission Process (CAP).f). Applicant should not avail any other Maintenance Allowance Scheme.g). For current Academic Yea

In [ ]:
# 